In [1]:
import os, shlex
from subprocess import check_output
repo_rootdir = check_output(shlex.split("git rev-parse --show-toplevel")).strip().decode('ascii')
os.chdir(repo_rootdir)

from random import shuffle
import numpy as np
import ipywidgets as widgets
import yaml
from copy import deepcopy
import json
import re
import itertools
import numpy as np
import scipy

from scipy.spatial import procrustes as procrustes

import glob, os, json
import pandas as pd

#from VTKHelpers.VTKMesh import VTKObject as Mesh
#from cardiac_mesh import CardiacMesh
from sklearn.decomposition import PCA, IncrementalPCA as IPCA
import pickle as pkl
#from config.config_parser import read_default_config

In [2]:
from config.cli_args import CLI_args, overwrite_config_items
from config.load_config import load_yaml_config, to_dict

In [3]:
ref_config = load_yaml_config("config_files/config.yaml")

In [4]:
meshes = pkl.load(open(ref_config.dataset.filename, "rb"))

In [5]:
def load_procrustes_transforms(filename):    
    return pkl.load(open(filename, "rb"))   

In [6]:
procrustes_transforms = load_procrustes_transforms("data/cardio/procrustes_transforms_35k.pkl")

In [7]:
from data.DataModules import CardiacMeshPopulationDataset

In [8]:
meshes_dataset = CardiacMeshPopulationDataset(meshes, procrustes_transforms)

In [9]:
N = 5000

In [11]:
shuffled_ids = [x for x in range(len(meshes_dataset))]
shuffle(shuffled_ids)

In [12]:
ids = [meshes_dataset[i]["id"] for i, _ in enumerate(meshes_dataset)]
meshes_lst = [meshes_dataset[id]["s"].numpy() for id in ids]
meshes_np = np.array(meshes_lst)
flattened_meshes = meshes_np.reshape(len(meshes_dataset),-1)
output_dir = "output/PCA__LV__%s_samples" % N
flattened_meshes[shuffled_ids[:N],:].shape

(5000, 15660)

Center meshes around mean shape:

In [18]:
flattened_meshes -= flattened_meshes.mean(0)

In [19]:
perf_dfs = []
n_comps = 16

# meshes_lst, ids_lst = flattened_partition(cardiac_data, config["nTraining"])
# print(shuffled_ids)

for n in range(1, n_comps+1):
  
  print(n)
  
  #TODO: This doesn't have to be done again for each n, but I need to figure out how to do it.
  training_meshes = flattened_meshes[shuffled_ids[:N],:]
  pp = PCA(n).fit(training_meshes)    
    
  meshes_reduced = pp.transform(flattened_meshes)

  meshes_reconstructed = pp.inverse_transform(meshes_reduced)
  mse = ((flattened_meshes - meshes_reconstructed)**2).mean(axis=1)
  
  mse_shuffled = ((flattened_meshes - meshes_reconstructed[shuffled_ids,:])**2).mean(axis=1)
    
  perf_df = pd.DataFrame({
    "model_id": ["PCA__LV__{}_comps__{}_samples".format(str(n_comps), str(N))] * len(meshes_dataset),
    "subject_id": shuffled_ids,
    "subset": ["training"] * N + ["test"] * (len(meshes_dataset) - N),
    "mse": mse,
    "mse_shuffled": mse_shuffled,
    "n": [n] * len(meshes_dataset),      
  })
  
  perf_df.index = shuffled_ids
  perf_dfs.append(perf_df)
        

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [20]:
pd.concat(perf_dfs).to_csv("PCA__LV__16_comps__5000_samples__performance.csv", header=True, index=False)

In [21]:
z_df = pd.DataFrame(meshes_reduced)
z_df.columns = ["z" + str(z).zfill(3) for z in z_df.columns]
z_df["ID"] = list(np.array(ids)[shuffled_ids])
z_df.to_csv("PCA__LV__16_comps__5000_samples__latent_space.csv", header=True, index=False)